# Python Brno - Part 2B - Pandas

Notes:
- Next level Pandas
  - https://github.com/TomAugspurger/modern-pandas

### Pandas

- Provides high-performance, easy-to-use data structures and data analysis tools for the Python programming language.


- `pandas.DataFrame`
  - 2D size-mutable data structure with labeled row index and labeled column index.
- `pandas.Series`
  - 1D size-mutable data structure with labeled row index and labeled column.


- Fantastic documentation: http://pandas.pydata.org/pandas-docs/stable/

### The Titanic Survival Problem

Predict whether a passenger on the titanic will survive. 
- Input: information about each passenger
- Output: whether or not the passenger survived

The data we will use is located in the file `titanic_data.csv` and is similar to the set from https://www.kaggle.com/c/titanic/data





### Exploring the Titanic Data

Load the pandas library with alias `pd`

In [ ]:
import pandas as pd
pd.options.display.max_rows = 8

Load matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

Read the data csv file into a pandas DataFrame

In [ ]:
df = pd.read_csv('titanic_data.csv')

### Missing Data

In [ ]:
df.info()

### The components of a DataFrame

In [ ]:
df.values

In [ ]:
df.index

In [ ]:
df.columns

### More on groupby / stack / unstack

It's possible to groupby multiple variables.

What's the difference in the fare between 1st class and 3rd class for females and males?

In [ ]:
df.groupby(['Sex','Pclass'])['Fare'].mean()

Here we want the difference between the first row and third row and the difference between the fourth row and sixth row. It's ugly to write code specifically targeting these rows. Instead we can transform the data.

### Use `unstack` to pivot index labels into column labels

In [ ]:
df.groupby(['Sex','Pclass'])['Fare'].mean().unstack()

### Use `stack` to pivot column labels into index labels

In [ ]:
df.groupby(['Sex','Pclass'])['Fare'].mean().unstack().stack()

Now we just need to subtract the two columns to get our answer

In [ ]:
avg_fare_groupedby_sex_v_class = df.groupby(['Sex','Pclass'])['Fare'].mean().unstack()
avg_fare_groupedby_sex_v_class

In [ ]:
abs(avg_fare_groupedby_sex_v_class.loc[:,3] - avg_fare_groupedby_sex_v_class.loc[:,1])

### Exercise

What was the average age of females and males who survived?

### Setting

Suppose we want to store this calculation in a new column.

In [ ]:
avg_fare_groupedby_sex_v_class.loc[:,'abs_diff_1_3'] = abs(avg_fare_groupedby_sex_v_class.loc[:,3] - avg_fare_groupedby_sex_v_class.loc[:,1])
avg_fare_groupedby_sex_v_class

Question: Does this change our original dataframe?

### Sorting

In [ ]:
df.sort_index(axis=0, ascending=False)

In [ ]:
df.sort_values(by='Pclass')

### Missing Data

In [ ]:
df.info()

To clear out any rows with missing data call `dropna`

In [ ]:
df.dropna().info()

Unfortunately this removes 80% of the observations in our dataset.

In [ ]:
len(df.dropna()) / len(df)

Instead we choose reasonable filler values for missing data based on inference or statistics.

Suppose we knew for example that any unspecified Cabin data meant that the passengers were staying in the Dorm room.

In [ ]:
df.Cabin

In [ ]:
df.Cabin.fillna(value='Dorm')

### Exercise

Two rows are missing data in the Port (Embarked) column. Talk with your neighbors about which port would be most appropriate to replace the missing data and then execute the appropriate command.

1. Feed `df.Embarked.isnull()` into `df[  ]` as a filter see what the missing data rows are
2. Use `value_counts` to determine which values are the most common

### Manipulating data

- `apply` - execute function on a row / column of a DataFrame
  - row based: df.apply(fn, axis=0)   # default
  - row based: df.apply(fn, axis=1)
- `applymap` - execute function elementwise on a DataFrame
- `map` - execute function elementwise on a Series

In [ ]:
import numpy as np

In [ ]:
df_age_fare = df[['Age','Fare']]
df_age_fare.describe()

Suppose we want to normalize some data between the values of 0 and 1. We can use a lambda function and `apply`

In [ ]:
df_age_fare = df[['Age','Fare']]
df_norm_1 = df_age_fare.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
df_norm_1.describe()

We can declare a separate function and pass it to `apply`

In [ ]:
df_age_fare = df[['Age','Fare']]

def my_norm(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

df_norm_2 = df_age_fare.apply(my_norm)
df_norm_2.describe()

Or we can use pandas built in functions to get the same result

In [ ]:
df_norm_3 = (df_age_fare - df_age_fare.min()) / (df_age_fare.max() - df_age_fare.min())

In [ ]:
df_norm_3.describe()

In [ ]:
df_age_fare.hist();
df_age_norm_3.hist();

### Exercise

1. Determine how many rows are missing from the Age column
2. Plot the histogram of Age
3. It's common practice to fill in missing data with the mean of the variable.
4. Create a new column called age_filled_with_mean and set it's value to Age with missing items replaced by the mean
5. Plot the histogram of Age vs the histogram of age_filled_with_mean
6. Discuss with your neighbour whether this is a good or bad approach to filling in the missing data

### Time Series / rolling functions

In [ ]:
# http://stackoverflow.com/questions/16734621/random-walk-pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def geometric_brownian_motion(T = 1, N = 100, mu = 0.1, sigma = 0.01, S0 = 20):        
    dt = float(T)/N
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(size = N) 
    W = np.cumsum(W)*np.sqrt(dt) ### standard brownian motion ###
    X = (mu-0.5*sigma**2)*t + sigma*W 
    S = S0*np.exp(X) ### geometric brownian motion ###
    return S

dates = pd.date_range('2012-01-01', '2016-02-22')
T = (dates.max()-dates.min()).days / 365
N = dates.size
start_price = 100
y = pd.Series(geometric_brownian_motion(T, N, sigma=0.1, S0=start_price), index=dates)
y.plot()
# plt.show()

In [ ]:
type(y.index)

In [ ]:
y

Since we are using a DatetimeIndex we can slice it based on month and year

In [ ]:
y.loc['2014-10']

In [ ]:
y.loc['2014']

To calculate a moving average we use rolling

In [ ]:
y.rolling(window=30).mean().plot()

In [ ]:
y.plot()
y.rolling(window=30).mean().plot()

### Exercise

Write a function which takes a DataFrame and a list of moving averages and returns the DataFrame with each of the moving averages calculated in a separate column

In [ ]:
moving_averages = [30, 60, 200]

def calculate_moving_averages(df, moving_average_list):

    # Fill me in
    
    return df

# Uncomment the next line
# calculate_moving_averages(y, moving_averages)